## Import Dependencies

In [40]:
import requests
import math
import pandas as pd

## Create functions to get and process data from dbt Cloud

In [61]:
def get_number_of_dbt_models_in_project(dbt_cloud_token, dbt_cloud_project_env_id):
    

    # Your GraphQL endpoint
    url = 'https://metadata.cloud.getdbt.com/graphql'
    
    # The GraphQL query
    query = """
            query Models($first: Int!, $environmentId: BigInt!) {
          environment(id: $environmentId) {
            applied {
              models(first: $first) {
                totalCount
              }
            }
          }
        }
    """

    # The variables to be used in your query
    variables = {
        "environmentId": dbt_cloud_project_env_id,
        "first": 500
    }
        
    # Headers, including possibly needed authorization
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {dbt_cloud_token}',  # Only include this line if you need authorization
    }
    
    # The payload for the request
    payload = {
        'query': query,
        'variables': variables
    }
    
    # Make the POST request to the GraphQL API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check for errors
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()

        # get number of models
        number_of_models_in_cloud_project = data['data']['environment']['applied']['models']['totalCount']

    else:
        print(f"Query failed to run by returning code of {response.status_code}. {response.text}")
    
    try:
        # return number of models in project
        return number_of_models_in_cloud_project
    except:
        print("there was an error getting back GraphQL Payload, please check your inputs")


def get_model_info_from_project(dbt_cloud_token, dbt_cloud_project_env_id, number_of_models_in_project):
    
    # Your GraphQL endpoint
    url = 'https://metadata.cloud.getdbt.com/graphql'
    
    # The GraphQL query
    query = """
            query Node($first: Int!, $environmentId: BigInt!, $after: String) {
              environment(id: $environmentId) {
                applied {
                  models(first: $first, after: $after) {
                    edges {
                      node {
                        tests {
                          name
                          resourceType
                        }
                        schema
                        materializedType
                        modelingLayer
                        uniqueId
                      }
                    }
                    pageInfo {
                      endCursor
                      hasNextPage
                      startCursor
                    }
                  }
                }
              }
            }
    """

    # Headers, including possibly needed authorization
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {dbt_cloud_token}',  # Only include this line if you need authorization
    }

    # calculate the number of API calls to make
    number_of_apis_to_make = math.ceil(number_of_models_in_project/500)

    # set page start
    page_start = None

    # put the parsed payload in a list
    parsed_model_data_list = []

    # loop through and make the calls
    for call in range(number_of_apis_to_make):

        # The variables to be used in your query
        variables = {
            "environmentId": dbt_cloud_project_env_id,
            "first": 500,
            "after": page_start
        }
    
        # The payload for the request
        payload = {
            'query': query,
            'variables': variables
        }
        
        # Make the POST request to the GraphQL API
        response = requests.post(url, json=payload, headers=headers)

        # get the data 
        data = response.json()['data']

        # parse the data
        parsed_data = data['environment']['applied']['models']

        # get the next page
        page_start = parsed_data['pageInfo']['endCursor']

        # add the parsed data to the list
        parsed_model_data_list.append(parsed_data['edges'])

    return parsed_model_data_list


def parse_model_data_into_a_csv(parsed_model_data_list):

    # 
    # Your data: a list of lists of dictionaries
    data_to_process = listx
    
    node_data = {}
    
    for sublist in data_to_process:
        for item in sublist:
            node = item['node']
            # Initialize or update the node entry in node_data
            if node['uniqueId'] not in node_data:
                node_data[node['uniqueId']] = {
                    'schema': node['schema'],
                    'materializedType': node['materializedType'],
                    'modelingLayer': "other" if node['modelingLayer'] is None else node['modelingLayer'],
                    'uniqueId': node['uniqueId'],
                    'tests': []  # Initialize an empty list to hold tests
                }
            # Append test names to the tests list if tests are present
            for test in node.get('tests', []):
                node_data[node['uniqueId']]['tests'].append(test['name'])
    
    # Convert the node_data dictionary to a list of dictionaries suitable for DataFrame creation
    flattened_data = list(node_data.values())

    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(flattened_data)

    # return the data frame
    return df

# Using the functions

### Set inputs

In [16]:
# get the dbt cloud service token or personal token
dbt_cloud_token = '<< dbt cloud token goes here >>'

# the production environment id for the dbt Cloud project 
dbt_cloud_project_env_id = 123456

### Get the number of dbt Models in the prod enviroment

In [63]:
# get the number 
number_of_models = get_number_of_dbt_models_in_project(dbt_cloud_token, dbt_cloud_project_env_id)

# log it
print(f"the number of dbt models in the env {dbt_cloud_project_env_id} is {number_of_models}")

the number of dbt models in the env 105436 is 40


### Get all of the info for the models in the prod env

In [65]:
# pull back data from discovery API
prod_env_model_info = get_model_info_from_project(dbt_cloud_token, dbt_cloud_project_env_id, number_of_models)

# log that it completed
print("grabbed all of the model info from the env")

grabbed all of the model info from the env


### Parse the list info a pandas dataframe

In [67]:
df_of_model_info = parse_model_data_into_a_csv(prod_env_model_info)

### Save the dataframe to csv

In [68]:
df_of_model_info.to_csv('./dbt_models_with_info.csv', index=False)

### Display the Dataframe

In [69]:
display(df_of_model_info)

,schema,materializedType,modelingLayer,uniqueId,tests
0,Curated_sample,view,other,model.primary_project.a_sample_model,[]
1,Curated,view,marts,model.primary_project.a_small_view_model,"[not_null_a_small_view_model_one, not_null_a_s..."
2,Curated,view,other,model.primary_project.consume_em,[]
3,Curated,view,other,model.primary_project.customer_dim_cleaned,[]
4,Curated,view,other,model.primary_project.customer_nations_in_nort...,[]
5,Curated,view,other,model.primary_project.customer_nations.v1,"[not_null_customer_nations_v1_nation_key, uniq..."
6,Curated,table,other,model.primary_project.customer_nations.v2,"[not_null_customer_nations_v2_nation_key, uniq..."
7,Curated,table,other,model.primary_project.customer_nations.v3,"[not_null_customer_nations_v3_nation_key, uniq..."
8,Curated,table,other,model.primary_project.customer_purchases,[]
9,Curated,view,marts,model.primary_project.dim_customers_ut,[]


___

# END OF SCRIPT

___